<a href="https://colab.research.google.com/github/Yugi-Seong/CNN/blob/main/Lec02_WideDeepRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Codes are adapted from SungKim@HKUST

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
cd '/content/drive/MyDrive/CNN'

/content/drive/MyDrive/CNN


## 01. Wide and Deep Model (Diabets_logistic)

In [ ]:
import torch
from torch.autograd import Variable
import numpy as np

In [ ]:
xy = np.loadtxt('data/diabetes.csv.gz', delimiter=',', dtype=np.float32)
x_data = Variable(torch.from_numpy(xy[:, 0:-1]))
y_data = Variable(torch.from_numpy(xy[:, [-1]]))

print(x_data.data.shape)
print(y_data.data.shape)

torch.Size([759, 8])
torch.Size([759, 1])


In [ ]:
class Model(torch.nn.Module):

    def __init__(self): # train 하고자 하는 weight의 모듈이 존재
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(Model, self).__init__()
        self.l1 = torch.nn.Linear(8, 5)
        self.l2 = torch.nn.Linear(5, 4)
        self.l3 = torch.nn.Linear(4, 1)

        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        out1 = self.sigmoid(self.l1(x))
        out2 = self.sigmoid(self.l2(out1))
        y_pred = self.sigmoid(self.l3(out2))
        return y_pred

# our model
model = Model()

In [ ]:
# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.

criterion = torch.nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [ ]:
# Training loop
for epoch in range(100):
        # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x_data)

    # Compute and print loss
    loss = criterion(y_pred, y_data)
    print(epoch, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 0.6579359769821167
1 0.656880795955658
2 0.6559155583381653
3 0.6550320982933044
4 0.6542235612869263
5 0.6534832119941711
6 0.6528052091598511
7 0.6521841287612915
8 0.6516149640083313
9 0.6510933637619019
10 0.6506152749061584
11 0.6501768827438354
12 0.6497747898101807
13 0.6494059562683105
14 0.6490675806999207
15 0.6487571001052856
16 0.6484720706939697
17 0.6482104659080505
18 0.6479702591896057
19 0.6477496027946472
20 0.6475470066070557
21 0.6473608016967773
22 0.6471897959709167
23 0.647032618522644
24 0.6468881964683533
25 0.6467553377151489
26 0.6466332077980042
27 0.6465209722518921
28 0.6464176177978516
29 0.6463226675987244
30 0.6462351679801941
31 0.6461548209190369
32 0.6460808515548706
33 0.6460126638412476
34 0.6459499597549438
35 0.6458922028541565
36 0.6458390355110168
37 0.6457900404930115
38 0.6457449197769165
39 0.6457033157348633
40 0.6456649303436279
41 0.6456296443939209
42 0.6455970406532288
43 0.645566999912262
44 0.6455393433570862
45 0.6455137133598328
4

## 02. Diabet Classification Using Custom DataLoader

In [ ]:
# References
# https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/pytorch_basics/main.py
# http://pytorch.org/tutorials/beginner/data_loading_tutorial.html#dataset-class
import torch
import numpy as np
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [ ]:
class DiabetesDataset(Dataset):
    """ Diabetes dataset."""

    # Initialize your data, download, etc.
    def __init__(self):
        xy = np.loadtxt('./data/diabetes.csv.gz',
                        delimiter=',', dtype=np.float32)
        self.len = xy.shape[0]
        self.x_data = torch.from_numpy(xy[:, 0:-1])
        self.y_data = torch.from_numpy(xy[:, [-1]])

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [ ]:
dataset = DiabetesDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=32,
                          shuffle=True,
                          num_workers=2)

In [ ]:
class Model(torch.nn.Module):

    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(Model, self).__init__()
        self.l1 = torch.nn.Linear(8, 6)
        self.l2 = torch.nn.Linear(6, 4)
        self.l3 = torch.nn.Linear(4, 1)

        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        out1 = self.sigmoid(self.l1(x))
        out2 = self.sigmoid(self.l2(out1))
        y_pred = self.sigmoid(self.l3(out2))
        return y_pred

# our model
model = Model()

In [ ]:
# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = torch.nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [ ]:
# Training loop
for epoch in range(2):
    for i, data in enumerate(train_loader, 0): #dataloader = Iterator
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(inputs)

        # Compute and print loss
        loss = criterion(y_pred, labels)
        print(epoch, i, loss.item())

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

0 0 0.6622541546821594
0 1 0.6104357838630676
0 2 0.6330622434616089
0 3 0.6764484643936157
0 4 0.5589278936386108
0 5 0.6448723673820496
0 6 0.6947207450866699
0 7 0.6288655996322632
0 8 0.59519362449646
0 9 0.6793532967567444
0 10 0.6614611744880676
0 11 0.6100084781646729
0 12 0.6260972023010254
0 13 0.6436702609062195
0 14 0.6618838906288147
0 15 0.6619544625282288
0 16 0.6800180673599243
0 17 0.6437458992004395
0 18 0.5546901822090149
0 19 0.6815656423568726
0 20 0.7374595403671265
0 21 0.6616527438163757
0 22 0.748808741569519
0 23 0.5800150036811829
1 0 0.6442949771881104
1 1 0.6440621614456177
1 2 0.5375999808311462
1 3 0.6238345503807068
1 4 0.6429299116134644
1 5 0.6036394834518433
1 6 0.5825416445732117
1 7 0.6016956567764282
1 8 0.6649593114852905
1 9 0.6642234921455383
1 10 0.6639039516448975
1 11 0.6224679946899414
1 12 0.622360110282898
1 13 0.6225181221961975
1 14 0.6430814266204834
1 15 0.6219834685325623
1 16 0.6433320045471191
1 17 0.6645122170448303
1 18 0.643157958

## 03. Softmax Loss

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [ ]:
# Cross entropy example
import numpy as np
# One hot
# 0: 1 0 0
# 1: 0 1 0
# 2: 0 0 1
Y = np.array([1, 0, 0])

Y_pred1 = np.array([0.7, 0.2, 0.1])
Y_pred2 = np.array([0.1, 0.3, 0.6])
print("loss1 = ", np.sum(-Y * np.log(Y_pred1)))
print("loss2 = ", np.sum(-Y * np.log(Y_pred2)))

loss1 =  0.35667494393873245
loss2 =  2.3025850929940455


In [ ]:
# Softmax + CrossEntropy (logSoftmax + NLLLoss)
loss = nn.CrossEntropyLoss()

In [ ]:
# target is of size nBatch
# each element in target has to have 0 <= value < nClasses (0-2)
# Input is class, not one-hot
Y = Variable(torch.LongTensor([0]), requires_grad=False)

In [ ]:
# input is of size nBatch x nClasses = 1 x 4
# Y_pred are logits (not softmax)
Y_pred1 = Variable(torch.Tensor([[2.0, 1.0, 0.1]]))
Y_pred2 = Variable(torch.Tensor([[0.5, 2.0, 0.3]]))

In [ ]:
l1 = loss(Y_pred1, Y)
l2 = loss(Y_pred2, Y)

print("PyTorch Loss1 = ", l1.data, "\nPyTorch Loss2=", l2.data)

print("Y_pred1=", torch.max(Y_pred1.data, 1)[1])
print("Y_pred2=", torch.max(Y_pred2.data, 1)[1])

PyTorch Loss1 =  tensor(0.4170) 
PyTorch Loss2= tensor(1.8406)
Y_pred1= tensor([0])
Y_pred2= tensor([1])


In [ ]:
# target is of size nBatch
# each element in target has to have 0 <= value < nClasses (0-2)
# Input is class, not one-hot
Y = Variable(torch.LongTensor([2, 0, 1]), requires_grad=False)

# input is of size nBatch x nClasses = 2 x 4
# Y_pred are logits (not softmax)
Y_pred1 = Variable(torch.Tensor([[0.1, 0.2, 0.9],
                                 [1.1, 0.1, 0.2],
                                 [0.2, 2.1, 0.1]]))


Y_pred2 = Variable(torch.Tensor([[0.8, 0.2, 0.3],
                                 [0.2, 0.3, 0.5],
                                 [0.2, 0.2, 0.5]]))


In [ ]:
l1 = loss(Y_pred1, Y)
l2 = loss(Y_pred2, Y)

print("Batch Loss1 = ", l1.data, "\nBatch Loss2=", l2.data)

Batch Loss1 =  tensor(0.4966) 
Batch Loss2= tensor(1.2389)


## 04.Softmax Classifier Using MNIST


In [3]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [4]:
# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)  # data 폴더 안에 MNIST dataset이 다운로드 됨 

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Processing...


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [6]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784) -> fully connected layer 에 넣음 #batch size 또는 64 대신 -1 을 입력해도 알아서 64가 입력됨
        # x = x.view(64, -1) 
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)


model = Net()

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [8]:
def train(epoch):
    model.train() # gradient 모두 사용
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)

        output = model(data)
        loss = criterion(output, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [12]:
def test():
    model.eval() #gradient 계산 안함 
    test_loss = 0
    correct = 0 #정확도 
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)

        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1] # max함수를 이용해 최댓값을 찾아 냄 
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [10]:
for epoch in range(1, 10):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300640
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.305988
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.302471
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.303741
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.295113
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.305640
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.299979
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.300575
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.306004
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.299477
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.303007
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.304554
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.292244
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.296706
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.298670
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.303335
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.305578
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.298925
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.296420
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.296590
Train Epoch: 1 [12800/60000 (

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  



Test set: Average loss: 0.0260, Accuracy: 5049/10000 (50%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.559411
Train Epoch: 2 [640/60000 (1%)]	Loss: 1.605780
Train Epoch: 2 [1280/60000 (2%)]	Loss: 1.538361
Train Epoch: 2 [1920/60000 (3%)]	Loss: 1.530875
Train Epoch: 2 [2560/60000 (4%)]	Loss: 1.697045
Train Epoch: 2 [3200/60000 (5%)]	Loss: 1.499442
Train Epoch: 2 [3840/60000 (6%)]	Loss: 1.295656
Train Epoch: 2 [4480/60000 (7%)]	Loss: 1.225676
Train Epoch: 2 [5120/60000 (9%)]	Loss: 1.363967
Train Epoch: 2 [5760/60000 (10%)]	Loss: 1.250846
Train Epoch: 2 [6400/60000 (11%)]	Loss: 1.391176
Train Epoch: 2 [7040/60000 (12%)]	Loss: 1.277794
Train Epoch: 2 [7680/60000 (13%)]	Loss: 1.285415
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.982104
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.933523
Train Epoch: 2 [9600/60000 (16%)]	Loss: 1.122081
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.929894
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.895440
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.789926
Train Epoch: 2 [12

## 03. Toy Inception MNIST

In [14]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [15]:
# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [ ]:
class InceptionA(nn.Module):

    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3dbl_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)

        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5)

        self.incept1 = InceptionA(in_channels=10)
        self.incept2 = InceptionA(in_channels=20)

        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incept1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incept2(x)
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)

In [ ]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [ ]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
for epoch in range(1, 10):
    train(epoch)
    test()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298459
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.302282
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.314808
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.313843
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.307911
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.301989
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.292517
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.291605
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.301653
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.293882
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.293476
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.288318
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.307489
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.282997
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.300429
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.278991
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.284487
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.271263
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.272692
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.256499
Train Epoch: 1 [12800/60000 (

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.1995, Accuracy: 9402/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.258538
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.217049
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.169010
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.160412
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.413642
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.295826
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.261276
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.092610
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.242168
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.269855
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.191170
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.146693
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.315070
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.197020
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.235060
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.152280
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.170992
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.221109
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.164026
Train Epoch: 2 [12